In [ ]:
import pandas as pd

# 파일 경로
file_path = "/content/dialogues_train.txt"

# 한 줄 = 하나의 대화 → 리스트로 저장
with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

# 전처리
data = []
for line in lines:
    utterances = line.strip().split("__eou__")
    utterances = [utt.strip() for utt in utterances if utt.strip()]  # 빈 발화 제거
    paragraph = " ".join(utterances)  # 문단으로 합치기

    data.append({
        "text": paragraph,
        "labels": 0,
        "category": "대화문"
    })

# DataFrame 생성
df = pd.DataFrame(data)

# CSV 저장
df.to_csv("dailydialog_human_from_txt.csv", index=False, encoding="utf-8-sig")
print("✅ CSV 저장 완료: dailydialog_human_from_txt.csv")

✅ CSV 저장 완료: dailydialog_human_from_txt.csv


In [ ]:
# prompt: dialog_totalcount.csv와 dailydialog_human_from_txt.csv를 합쳐주는  코드 작성해줘. 근데 labels에 값은 그대로 작성된대로 합쳐줘. 그리고 dialog_totalcount.csv데이터 밑에 행으로 dailydialog_human_from_txt.csv 데이터가 들어가서 하나의 csv 파일로 저장되게 해줘.
# Total_Count에 값이 없는 것은 null값으로 채워줘

# Load the two CSV files
df_totalcount = pd.read_csv("dialog_totalcount.csv")
df_dailydialog = pd.read_csv("dailydialog_human_from_txt.csv")

# Ensure both dataframes have the same columns, filling missing 'Total_Count' with None in the second dataframe
if 'Total_Count' not in df_dailydialog.columns:
  df_dailydialog['Total_Count'] = None

# Reorder columns to ensure consistency before concatenation
df_dailydialog = df_dailydialog[df_totalcount.columns]


# Concatenate the dataframes row-wise
combined_df = pd.concat([df_totalcount, df_dailydialog], ignore_index=True)

# Save the combined dataframe to a new CSV file
combined_df.to_csv("combined_dialogues.csv", index=False, encoding="utf-8-sig")

print("✅ Combined CSV saved: combined_dialogues.csv")


✅ Combined CSV saved: combined_dialogues.csv


In [ ]:
# prompt: combined_dialogues.csv에서 아예 그 한 데이터 자체가 (문단 자체로) 똑같은 데이터값이 2개 이상 중복되는지 보여주는 코드 작성해줘.

# Check for duplicate rows based on the 'text' column
duplicate_rows = combined_df[combined_df.duplicated(subset=['text'], keep=False)]

# Print the duplicate rows if any
if not duplicate_rows.empty:
  print("\n✨ 중복된 데이터 (문단 자체):")
  print(duplicate_rows)
else:
  print("\n✨ 중복된 데이터가 없습니다.")



✨ 중복된 데이터 (문단 자체):
                                                    text  labels category  \
57328  Hey John , nice skates . Are they new ? Yeah ,...       0      대화문   
57337  Diana , do you like the perfume I gave you ? I...       0      대화문   
57343  Can I help you ? I hope so . I'm looking for s...       0      대화문   
57344  Here ’ s your hot dog and beer . What happened...       0      대화문   
57350  Do you like traveling , Kathy ? I like traveli...       0      대화문   
...                                                  ...     ...      ...   
68382  Hi , are you Catherine ? Yes , Jason , right ?...       0      대화문   
68389  Would you show me some table-cloths ? Certainl...       0      대화문   
68406  Mr . McKenna , do you have a second ? I need t...       0      대화문   
68413  I need to purchase some business cards . No pr...       0      대화문   
68433  Dick , I ’ Ve decided not to rent that house ....       0      대화문   

      Total_Count  
57328        None  
57337        No

In [ ]:
# prompt: combined_dialogues.csv에서 아예 그 한 데이터 자체가 (문단 자체로) 똑같은 데이터값이 2개 이상 중복되는지 보여주는 코드 작성해줘. 그리고 어떤 행과 행이 같은지 보여줘. 그리고 이 데이터들은 같은 데이터들 중 하나만 냅두고 나머지는 삭제하는 코드 부탁해

# Check for duplicate rows based on the 'text' column
duplicate_rows = combined_df[combined_df.duplicated(subset=['text'], keep=False)]

if not duplicate_rows.empty:
    print("✅ Duplicate rows found:")
    # Group by the duplicated text and show all rows for each duplicated text
    for text, group in duplicate_rows.groupby('text'):
        print(f"\nDuplicate text: {text}")
        print(group[['text', 'labels', 'category']])
else:
    print("✅ No duplicate rows found based on the 'text' column.")

# Remove duplicate rows, keeping only the first occurrence
combined_df_cleaned = combined_df.drop_duplicates(subset=['text'], keep='first')

# Save the cleaned dataframe to a new CSV file
combined_df_cleaned.to_csv("combined_dialogues_cleaned.csv", index=False, encoding="utf-8-sig")

print("\n✅ Duplicate rows removed and cleaned CSV saved: combined_dialogues_cleaned.csv")

✅ Duplicate rows found:

Duplicate text: A friend of mine is now in the big house . He was caught red-handed for selling cocaine . That is a deadly risky business . He should have it coming on him . He must be regretting about having done the business . I bet he is . He domed his own life .
                                                    text  labels category
58042  A friend of mine is now in the big house . He ...       0      대화문
59629  A friend of mine is now in the big house . He ...       0      대화문

Duplicate text: Ah ! It hurts . Don't touch it . What part hurts ? The shoulder . Well , maybe you broke it . But what I'm worried about is this cut . It's not a cut . It's a gash ! It's bigger than a cut ! Ah ! I need a doctor . C'mon . Just don't move . I'm bleeding too much . We don't have a decent First Aid Kit , do we ? Yes . My bike has one under the seat . Get it , quick ! There's some tape , iodine , and cloth bandages . I don't think the cloth bandages can stop the bleedi

In [ ]:
# prompt: combined_dialogues_cleaned.csv의 labels의 0과 1의 갯수와 차이를 보여줘

# Assuming 'combined_dialogues_cleaned.csv' is already loaded into a DataFrame named 'combined_df_cleaned'
# If not, load it:
# combined_df_cleaned = pd.read_csv("combined_dialogues_cleaned.csv")

# Get the count of 0s and 1s in the 'labels' column
label_counts = combined_df_cleaned['labels'].value_counts().sort_index()

print("Labels Count:")
print(label_counts)

# Calculate the difference between the count of 1s and 0s
if 0 in label_counts and 1 in label_counts:
    difference = abs(label_counts[1] - label_counts[0])
    print(f"\nDifference between count of 1 and 0: {difference}")
elif 0 in label_counts:
    print(f"\nDifference between count of 1 and 0: {label_counts[0]} (only 0s present)")
elif 1 in label_counts:
    print(f"\nDifference between count of 1 and 0: {label_counts[1]} (only 1s present)")
else:
    print("\nNo 0 or 1 labels found in the 'labels' column.")

Labels Count:
labels
0    20635
1    47234
Name: count, dtype: int64

Difference between count of 1 and 0: 26599


In [ ]:
!mkdir -p data/full_dataset/
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_3.csv

--2025-06-14 09:37:41--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.8.207, 142.251.170.207, 173.194.174.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.8.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14174600 (14M) [application/octet-stream]
Saving to: ‘data/full_dataset/goemotions_1.csv’

goemotions_1.csv    100%[===================>]  13.52M  8.68MB/s    in 1.6s    

2025-06-14 09:37:42 (8.68 MB/s) - ‘data/full_dataset/goemotions_1.csv’ saved [14174600/14174600]

--2025-06-14 09:37:43--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.8.207, 142.251.170.207, 173.194.174.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.8.207|:443... connected.
HTTP request sent, awaiting respo

In [ ]:
import pandas as pd
import glob

# 모든 CSV 경로 수집
files = glob.glob("data/full_dataset/goemotions_*.csv")

# 확인: 파일 제대로 읽히는지
print(files)  # 혹시 비어있으면 위 wget 셀 다시 실행해야 해!

# 파일이 존재한다면 병합
if files:
    df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
    print(f"✅ 총 샘플 수: {len(df)}")
    print(df.head())
else:
    print("❌ CSV 파일을 찾을 수 없습니다. 다운로드가 제대로 되었는지 확인하세요.")

['data/full_dataset/goemotions_2.csv', 'data/full_dataset/goemotions_1.csv', 'data/full_dataset/goemotions_3.csv']
✅ 총 샘플 수: 211225
                                                text       id  \
0                                        We can hope  ee3o3ko   
1                     Shhh don't give them the idea!  eebl3z7   
2  Thank you so much, kind stranger. I really nee...  ed4fe9l   
3  Ion know but it would be better for you to jus...  efavtdu   
4  I'm honestly surprised. We should have fallen ...  ee2imz2   

               author            subreddit    link_id   parent_id  \
0       darkenseyreth       EdmontonOilers  t3_ag4r9j  t1_ee3mhad   
1   BoinkBoinkEtAliae      MurderedByWords  t3_ah3o76  t1_eeb68lo   
2          savageleaf  raisedbynarcissists  t3_abwh00  t1_ed4etbj   
3           CADDiLLXC              darknet  t3_al4njw   t3_al4njw   
4  CorporalThornberry    CollegeBasketball  t3_afxt6t  t1_ee22nyr   

    created_utc  rater_id  example_very_unclear  admiration  .

In [ ]:
import pandas as pd
from collections import defaultdict
import ast

# df는 기존에 병합된 goemotions 데이터프레임
# text, labels 컬럼이 없다고 가정 (labels 컬럼이 없는 상태에서 감정 정보를 따로 어떻게 쓸지는 모르지만, 우선 'emotion' 컬럼이 있다고 가정하자)
# 예시로 'emotion' 컬럼에 단일 정수형 감정 인덱스가 있다고 가정하고 진행

# 만약 'emotion' 컬럼 없으면, 임의로 0~27 사이 임의 감정 레이블로 생성 예시 (실제 데이터 맞게 수정 필요)
import numpy as np
if 'emotion' not in df.columns:
    np.random.seed(42)
    df['emotion'] = np.random.randint(0, 28, size=len(df))

# 감정 인덱스별로 문장 묶기
emotion_sentences = defaultdict(list)

for _, row in df.iterrows():
    emotion = row['emotion']
    text = row['text'] if 'text' in df.columns else str(row[0])  # 혹시 text 컬럼 없으면 첫 컬럼으로 대체
    emotion_sentences[emotion].append(text)

# 3개 이상 문장 있는 감정만 필터
emotion_sentences_filtered = {k: v for k, v in emotion_sentences.items() if len(v) >= 3}

# 문단 생성 (문장 3개씩 묶기)
paragraphs = []
for emotion, sentences in emotion_sentences_filtered.items():
    for i in range(0, len(sentences), 3):
        chunk = sentences[i:i+3]
        if len(chunk) == 3:
            paragraph = " ".join(chunk)
            paragraphs.append(paragraph)

# 26599개로 자르기
paragraphs = paragraphs[:26599]

# 새로운 데이터프레임 생성
output_df = pd.DataFrame({
    'text': paragraphs,
    'labels': 0,
    'category': 'dialog'
})

# CSV 저장
output_df.to_csv('output_goemotions_paragraphs.csv', index=False)

print("✅ CSV 파일이 생성되었습니다: output_goemotions_paragraphs.csv")
print(output_df.head())
print(f"최종 데이터셋 크기: {len(output_df)}")

✅ CSV 파일이 생성되었습니다: output_goemotions_paragraphs.csv
                                                text  labels category
0  We can hope One of my worst nightmares. Ugh. j...       0   dialog
1  Is this a real question or a snark? We only wa...       0   dialog
2  Somebody is really insecure about their career...       0   dialog
3  a celebrity i like had his appendix out and tw...       0   dialog
4  > It's crazy you don't understand this. i r o ...       0   dialog
최종 데이터셋 크기: 26599


In [ ]:
# prompt: output_goemotions_paragraphs.csv랑, combined_dialogues_cleaned.csv를 합치는 코드 작성해줘. 근데, 합칠 때 Total_count가 비어있는데이터는 그대로 null값으로 비워서 합쳐줘. 그리고 아예 그 한 데이터 자체가 (문단 자체로) 똑같은 데이터값이 2개 이상 중복되는지 보여주는 코드 작성해줘. 그리고 어떤 행과 행이 같은지 보여줘. 그리고 이 데이터들은 같은 데이터들 중 하나만 냅두고 나머지는 삭제하는 코드 부탁해

# Load the two CSV files
df_goemotions = pd.read_csv("output_goemotions_paragraphs.csv")
df_combined_cleaned = pd.read_csv("combined_dialogues_cleaned.csv")

# Ensure both dataframes have the same columns.
# 'Total_Count' from df_combined_cleaned might be null, and that's acceptable for merging.
# We need to ensure df_goemotions has the same columns, adding 'Total_Count' if it's missing and filling with None.
required_columns = ['text', 'labels', 'category', 'Total_Count'] # Assuming these are the desired columns

for col in required_columns:
    if col not in df_goemotions.columns:
        df_goemotions[col] = None

# Reorder columns to ensure consistency before concatenation
df_goemotions = df_goemotions[required_columns]
df_combined_cleaned = df_combined_cleaned[required_columns]


# Concatenate the dataframes row-wise.
# The 'Total_Count' column from df_combined_cleaned will be preserved, including null values.
combined_final_df = pd.concat([df_goemotions, df_combined_cleaned], ignore_index=True)

# Save the combined dataframe to a new CSV file
combined_final_df.to_csv("combined_dialogues_final.csv", index=False, encoding="utf-8-sig")

print("✅ Final Combined CSV saved: combined_dialogues_final.csv")

# Check for duplicate rows based on the 'text' column
duplicate_rows_final = combined_final_df[combined_final_df.duplicated(subset=['text'], keep=False)]

# Print the duplicate rows if any
if not duplicate_rows_final.empty:
  print("\n✨ 중복된 데이터 (문단 자체) - 최종 병합본:")
  print(duplicate_rows_final)

  # Show which rows are duplicates of each other
  print("\n✅ Duplicate rows found based on the 'text' column in the final combined data:")
  # Group by the duplicated text and show all rows for each duplicated text
  for text, group in duplicate_rows_final.groupby('text'):
      print(f"\nDuplicate text: {text}")
      print(group[['text', 'labels', 'category', 'Total_Count']])
else:
    print("\n✨ 최종 병합본에 중복된 데이터가 없습니다.")

# Remove duplicate rows from the final combined dataframe, keeping only the first occurrence
combined_final_df_cleaned = combined_final_df.drop_duplicates(subset=['text'], keep='first')

# Save the cleaned final dataframe to a new CSV file
combined_final_df_cleaned.to_csv("combined_dialogues_final_cleaned.csv", index=False, encoding="utf-8-sig")

print("\n✅ Duplicate rows removed from final combined data and cleaned CSV saved: combined_dialogues_final_cleaned.csv")


<ipython-input-27-2028011241>:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_final_df = pd.concat([df_goemotions, df_combined_cleaned], ignore_index=True)


✅ Final Combined CSV saved: combined_dialogues_final.csv

✨ 최종 병합본에 중복된 데이터가 없습니다.

✅ Duplicate rows removed from final combined data and cleaned CSV saved: combined_dialogues_final_cleaned.csv


In [ ]:
# prompt: combined_dialogues_final_cleaned.csv의 labels의 0과 1의 갯수와 차이를 보여줘

# Load the final cleaned CSV
df_final_cleaned = pd.read_csv("combined_dialogues_final_cleaned.csv")

# Get the count of 0s and 1s in the 'labels' column
label_counts_final = df_final_cleaned['labels'].value_counts().sort_index()

print("Labels Count in combined_dialogues_final_cleaned.csv:")
print(label_counts_final)

# Calculate the difference between the count of 1s and 0s
if 0 in label_counts_final and 1 in label_counts_final:
    difference_final = abs(label_counts_final[1] - label_counts_final[0])
    print(f"\nDifference between count of 1 and 0 in combined_dialogues_final_cleaned.csv: {difference_final}")
elif 0 in label_counts_final:
    print(f"\nDifference between count of 1 and 0 in combined_dialogues_final_cleaned.csv: {label_counts_final[0]} (only 0s present)")
elif 1 in label_counts_final:
    print(f"\nDifference between count of 1 and 0 in combined_dialogues_final_cleaned.csv: {label_counts_final[1]} (only 1s present)")
else:
    print("\nNo 0 or 1 labels found in the 'labels' column of combined_dialogues_final_cleaned.csv.")

Labels Count in combined_dialogues_final_cleaned.csv:
labels
0    47234
1    47234
Name: count, dtype: int64

Difference between count of 1 and 0 in combined_dialogues_final_cleaned.csv: 0
